# 1:  setting up
Luckily for us as programmers and data scientists we don't have to write all our code from scratch.
There are a lot of code libraries and modules available for many of the common tasks we will need to carry out.
The first step, before we start looking at data is to load some of these modules so that we can call on them in our code.The modules we are going to import are:

- pandas: a popular general-purpose data analysis library which allows importing data from various file formats. It also provides useful data structures which can be used to examine and edit data

- numpy: a popular library for performing mathematical operations, particularly with matrices and arrays.

- matplotlib: a library for plotting charts and other visualisations.

- Seaborn: a library which uses matplotlib and provides extra functions for visualising statistical analysis results.

- toolbox_module: this contains some python functions we wrote to perform complex tasks which need very long blocks of code.
importing them means we don't have to write them in this notebook, making it easier for you to read. As you progress with
python you will do this a lot with your own code.

You will need to run the code in the next cell so that al the code that relies on these libraries can work.

In [ ]:
#Loading libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import toolbox_module as tools
import sklearn

# 2: Loading the Data
Now we are going to import the data set. It is in a .csv file in a folder called 'data' the same directory as this notebook.
We are making a variable called 'df'. We also have access to a variable 'pd' which we created in the previous cell when
we imported _pandas._ We are calling a function in pandas (pd) called 'read()' which uses the location of the data as a string argument to load a
dataset into a pandas _dataframe_ (think of this as a table)

In [ ]:
# load the dataset
df = pd.read_csv("data/diabetic_data.csv")

# Exploring the data
Now we have our data in a dataframe we can start examining and manipulating it.
Now let's explore the data. The first command we will use is in the cell below: `df.shape` this will return two values
in parentheses (in Python we call this a _tuple_). The first value is the number of rows in our dataset; the second is the
number of columns.

In [ ]:
df.shape

### In your Notebook...
1. How many rows of data are in this dataset?
2. Could you open this data easily in Excel?

The next command we will try is `.head()` By default this will display the first 5 lines of data. It is useful for
giving a general idea of what the data looks like. If you put a number in the brackets it will return that amount of data

### try...
Type a number into the brackets and re-run the cell (if you tried to be a clever-clogs and typed 1000 like I would have
done ;-) you can get rid of it by re-running the cell with a smaller number. 
If you type ````.T```` on the end it will flip the table view so the columns display as rows

In [ ]:
# type your code below
df.head(100)


#we could put the stacktrace bit in the getting started section....

If everything went OK you should see the first 5 rows of data. If not you will see a lot of red text called a _stack trace_.
Don't worry about this. It might look alarming but it often includes helpful information about where you went wrong.
If you do see a stack trace, make sure your code in the cell above looks like this `df.head()` once it does, and you
re-run the cell the stacktrace will be gone.
### try
replace `df.head()` with `df.hea()`
### in your Notebook
What did the output look like?  Look at it carefully. How might that help you work out
what went wrong?

now put df.head() back into the cell and re-run it.

The dataset should have 101766 rows, and 50 columns. That's a lot of data to make sense of, but we have the tools!
Let's see what types of data we have. The command to do this is `df.dtypes` try it in the cell below.

In [ ]:
df.dtypes

Next to each column you can see data type. for example _int64_ is an integer. When Pandas isn't sure what
the type should be it uses 'object'. These are mostly strings as this is data coming from a .CSV file.
Now we know what our data looks like lets make sure it is in a suitable state for us to analyse. One of the first things
to do is deal with any missing values.

In this dataset, missing values are represented by a  “?”  So if we search every column which has a type of “object” to
see if there are any “?” values, and then count them, we can see the extent of any missing data problem.

In [ ]:
# note to self - do we need to take them through series and referencing pandas dataframes ? Or can we just explain what it does

# for every column in the dataframe
for col in df.columns:
    # check if it has a datatype of 'object' - if it does
    if df[col].dtype == object:
        #print the column name and the count of the number of '?' values
         print(col,df[col][df[col] == '?'].count())

We know that 'Gender' is encoded differently with 'Unknown/Invalid' used to signify missing data.
so:

In [ ]:
tools.count_value(df, 'Unknown/Invalid', 'gender')

## Cleaning up the Data: Missing values.

We can see that some columns have quite a few missing values. Weight is the worst affected with around 98% of
the values missing. There is not much we can do to improve this situation so we will remove this column from the dataset.
Payer code and medical specialty also have a lot of missing values so we will also drop these columns. 2 other variables
the drugs named <drug names> all have the same values and therefore do not add any information, so we will also drop these

In [ ]:
# dropping columns with large number of missing values
df = df.drop(['weight','payer_code','medical_specialty'], axis = 1)

The columns race, diagnoses and gender all have some missing values but far fewer than the columns we have just dropped.
In this situation, we can just go in and delete those values. We will do this with a function from our toolbox module.
Once we have done this we can rerun the code we used earlier to see what our dataset looks like now we have cleaned it
up a bit

In [ ]:
tools.drop_values(df,'race','?')
tools.drop_values(df,'diag_1', '?')
tools.drop_values(df,'diag_2', '?')
tools.drop_values(df,'diag_3', '?')
tools.drop_values(df,'gender', 'Unknown/Invalid')

## Feature Engineering
So we have cleaned up the data and now have a dataset free from missing values, but there are still some issues.
The dataset wasn't designed specifically with machine learning in mind, and some of the variables need to be optimised.
There are also some potential new features an algorithm could use.

### Previous Diagnoses
There are three columns in the dataframe: ``diag1``, ``diag2`` and ``diag3`` which are references to previous diagnoses.
These use something called an IDC code to indicate what the diagnosis was.
There are several hundred distinct values which are strings, mostly numeric.
we can simplify this into something our model can make sense out of by categorising them. In another study, researchers
used nine desease categories and we have done the same, assigning each a single digit identifier:
circulatory = 1, respiratory = 2,digestive = 3,diabetes = 4,injury = 5,musculoskeletal = 6,
genitourinary = 7, neoplasm = 8 and other = 9

We have provided a function in the toolbox_module which does this for you: `convert_idc_disease_class()`
This accepts a string and returns an integer between 1 and 9.
#### try...
patient x has a diag_1 code of '648'. what type of condition did they suffer from?

In [ ]:
#write your code below...
tools.convert_idc_disease_class('648') #todo: delete me - I'm a solution

The three lines of code in the cell below apply this function to every row in the dataframe and put the result in a new
column. This uses a feature you will come across in many programming languages, called a _lambda function_. It works by
passing a bit of code as an argument to a function. The code then runs inside the function and enhances it.

You can see the Keyword `Lambda` and `row`followed by a colon. `row` is a parameter for the dataframe's `apply` function.
After the colon we pass our toolbox function which is applied to every row in the dataframe.

In [ ]:
df['diag_1_cat']=df.apply(lambda row:tools.convert_idc_disease_class(row['diag_1']), axis=1)
df['diag_2_cat']=df.apply(lambda row:tools.convert_idc_disease_class(row['diag_2']), axis=1)
df['diag_3_cat']=df.apply(lambda row:tools.convert_idc_disease_class(row['diag_3']), axis=1)

Now lets run df head again. Can you see what's changed?

In [ ]:
df.head(100).T

### Drug prescribing
There are 23 drugs which have been prescribed as part of diabetes treatment protocols and the dataset records whether
the prescription was increased, reduced, not prescribed, or continued without change.
Have a look at the `head()` output to see them.
We are aware of previous research which has found that changes to medication is associated with lower readmission rates,
so we are going to make a new variable - a *__'feature'__* for our machine learning algorithm called ``numchange`` which
is the total number of changes to any medication.

To do that we make a list of all the medications (keys), and then we loop over this making a temporary column in the
dataframe for each key and then using a *__lambda__* function like we did earlier. This creates a variable “X” which stores
the value of the column, which we test: if it is equal to 'no' or 'steady', then the medication either has not been
prescribed or dosage not changed in which case we assign 0. If it has any other value, i.e. “up”, or “down” then the
dosages either been increased or decreased, in which case we assign one. 

Once we have done that we loop over the keys again, this time adding the value of the temporary column to the `numchange` column. Finally we delete the temporary column. The last line of code outputs the number of different values in the column

In [ ]:
# set up the list of drugs - the columns we are using
keys = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide',
        'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'insulin', 'glyburide-metformin', 'tolazamide',
        'metformin-pioglitazone','metformin-rosiglitazone', 'glimepiride-pioglitazone', 'glipizide-metformin',
        'troglitazone', 'tolbutamide', 'acetohexamide']

##loop over the drug columns, and for every drug...
for col in keys:
    #... make a temporary column
    colname = str(col) + 'temp'
    #...then check for changes to medications: assign 1 to the temporary column if there were changes, otherwise 0
    df[colname] = df[col].apply(lambda x: 0 if (x == 'No' or x == 'Steady') else 1)

#make a numchange column: we do this outside the loop, why?
df['numchange'] = 0

# now we loop over them again
for col in keys:
    #remake the name of the temporary column
    colname = str(col) + 'temp'

    # add the value to the numchange column
    df['numchange'] = df['numchange'] + df[colname]
    # delete the temorary column
    del df[colname]

#print out a list of all the 'numchange' values we have and how many
df['numchange'].value_counts()


### In your notebook...
What percentage of people did not have changes to their medications?
## Encoding
The dataset uses strings for gender, race, diabetesMed, and medication change, and for each of the 23 drugs used. To use these values in our
model we are going to convert them into binary values, i.e. 1 and 0.
#### Medication Change
First, we will deal with medication change by replacing ‘Ch’ with 1 and ‘No’ with 0

In [ ]:
df['change'] = df['change'].replace('Ch', 1)
df['change'] = df['change'].replace('No', 0)

#### diabetesMed
...then we will change diabetesMed by replacing yes with 1 and no with 0. This is a convention in programming when
converting things into binary. 'yes' is generally represented by '1' and 'no' generally by '0', a bit like a Boolean value,
where 'True' is represented by '1' and 'False' by '0'

In [ ]:
df['diabetesMed'] = df['diabetesMed'].replace('Yes', 1)
df['diabetesMed'] = df['diabetesMed'].replace('No', 0)

### try...
#### Gender
Now it is your turn. Let’s encode gender by encoding 'Male' with 1 and 'Female' with 0. You can do this in tha same way
as we did in the cells above Have a look at the dataset to make sure that you type 'Male' and 'Female' exactly as they are
in the dataset. Clue: both capitalised

In [ ]:
# write your code below...
df['gender'] = df['gender'].replace('Male', 1)
df['gender'] = df['gender'].replace('Female', 0)

#### Ethnicity
The column ‘ethnicity’ currently has text values to describe the ethnic background of patients. Typical values include
“African/American”, “Caucasian”, “Hispanic”, and “other”. Machine learning algorithms prefer numerical or binary values
so to improve our data, we will create a column for each of the text values and indicate the ethnicity by placing a “1”
in the correct column and “0” in all the others. These are known as “dummy variables”, and we are using the pandas dataframe
function ``get_dummies()`` in the cell below to create them.

In [ ]:
df = pd.get_dummies(df, columns=['race'])

#### Medications
The dataset contains 23 columns each for a different medication. Each medication feature indicates whether the drug was
prescribed and whether there was a change in dosage.As we have already created a variable that measures the amount of
changes in medication alongside the existence of another variable which indicates whether or not there was a change, we
don't really need this information, so we are going to recode these variables so that they just indicate whether or not
this particular drug was prescribed. We are going to loop through the set of drug keys we created earlier. This time,
for each drug, we will replace the values: 0 for ‘no’;1 for steady, up, down

In [ ]:
# we are using the list of keys from the cell we created them in earlier
for col in keys:
    df[col] = df[col].replace('No', 0)
    df[col] = df[col].replace('Steady', 1)
    df[col] = df[col].replace('Up', 1)
    df[col] = df[col].replace('Down', 1)

### Number of medications
Number of each medication used: Another potentially interesting factor could be the total number of medications used by the
patient. This might indicate the severity of their condition and/or the intensity of their care, so we created another
feature - num_meds by counting the medications used during the stay.

In [ ]:
# make a new column
df['num_meds'] = 0
# loop through the list of drugs we made earlier
for col in keys:
    # there will now be a '1' in the column if the patient is on that drug, so add it to the number of meds
    # (add the '0' otherwise)
    df['num_meds'] = df['num_meds'] + df[col]
# print a count of the values to make sure it worked
df['num_meds'].value_counts()

#### Test Results
We are also going to recode test results for the A1c test (we are data scientists and we have no idea what that is), and
the maximum glucose serum test. We are going to encode 0 for a normal result and 1 for an abnormal result.
Where no test was given, we will use -99.
#### in your notebook...
Have a look at the code in the cell below, what are the abnormal values for each test?

In [ ]:
df['A1Cresult'] = df['A1Cresult'].replace('>7', 1)
df['A1Cresult'] = df['A1Cresult'].replace('>8', 1)
df['A1Cresult'] = df['A1Cresult'].replace('Norm', 0)
df['A1Cresult'] = df['A1Cresult'].replace('None', -99)
df['max_glu_serum'] = df['max_glu_serum'].replace('>200', 1)
df['max_glu_serum'] = df['max_glu_serum'].replace('>300', 1)
df['max_glu_serum'] = df['max_glu_serum'].replace('Norm', 0)
df['max_glu_serum'] = df['max_glu_serum'].replace('None', -99)

### Age Groupings
The age variable is currently encoded in 10 year intervals: 10 year bands from 0 to 100. We are going to create a loop
which runs 10 times and replaces each band with an integer. What we are left with is still a categorical variable, so we
will approximate the age as the midpoint in each category, so we are left with something on a continuous scale.

To do this we use a feature in python called a dictionary. This is a data structure which is a type of *__collection__*. It's a  a bit like a list, but it consists of key value pairs separated by a colon. The key comes first, and we can use this to look up values in the dictionary in much the same way you would get the definition of a word from a real-life dictionary.

In [ ]:
# code age intervals [0-10) - [90-100) from 1-10
for i in range(0,10): #for every number from 1-10
    #make a string using i which matches the interval eg '[10-20)', replace this with i+1
    df['age'] = df['age'].replace('['+str(10*i)+'-'+str(10*(i+1))+')', i+1)
df['age'].value_counts()

#convert 'age' into an integer
df['age'] = df['age'].astype('int64')
#print out a count of all the values. This keep us sane!
print(df.age.value_counts())

# make a dictionary so we can look up new values to replace the ones we made earlier
age_dict = {1:5, 2:15, 3:25, 4:35, 5:45, 6:55, 7:65, 8:75, 9:85, 10:95}

#map the values onto those in the dictionary. This will replace each one
df['age'] = df.age.map(age_dict)
#print out the new value counts so we know it worked
print(df.age.value_counts())


### Encoding the outcome variable
The outcome variable is the thing we are trying to predict. As it currently stands it has the values 'no', '>30' and
'<30' representing not readmitted, readmitted in less than 30 days and readmitted after more than 30 days.
You can see the current values by running ``value_counts()`` in the cell below.

In [ ]:
df['readmitted'].value_counts()

We are going to encode the outcome variable so that readmission within 30 days is indicated as a 1 and no readmission or
readmission later than 30 days is indicated as a 0. We will do this with the pandas replace function in the cell below.

In [ ]:
df['readmitted'] = df['readmitted'].replace('>30', 0)
df['readmitted'] = df['readmitted'].replace('<30', 1)
df['readmitted'] = df['readmitted'].replace('NO', 0)

### Visualisations
Another important step in a machine learning project is visual inspection of the data. We had a look at the output
from the `head()` command, but as humans we can only make sense of this one column at a time, so we can only use it to
inspect a tiny fraction of the data. We need other strategies to get a picture of what the whole dataset looks like,
so let’s build some charts…

Remember those imports we did at the start of this notebook? One of them was *__Seaborn__* which is a statistical
visualisation library. We imported it as `sns`. This means we can use it to produce charts by calling its functions.
The seaborne library is based on *__matplotlib__* and uses many of its functions, so when we write code, we might call
functions from matplotlib (`plt`) as well as seaborne in the same chart.

Let's start with a nice simple one and plot _age_ and _readmission_ as a bar chart...

#### Age and Readmission
First, we create a `fig` variable by calling the `figure` method from `plt` which is a quick way of setting the size of the graphic.
Any matplotlib or seaborne code we call now will operate on this fig variable until we make a new one.
Next we call `sns.countplot()` to create a simple bar chart counting the values of a categorical variable using a column
from the data frame as the first argument (`y=`) to measure on the x-axis, and then use the column supplied as the `hue` argument,
which groups and colours the bars along the y-axis according to a second categorical variable.
### try...
See if you can flip the chart around by changing the name of the first argument. Which looks better?

In [ ]:
fig = plt.figure(figsize=(15,10))
chart = sns.countplot(y= df['age'], hue = df['readmitted'])
chart.set_title('Age of Patient VS. Readmission')

Now for a slightly more complex example. This next chart plots time in hospital and frequency of those readmitted, and
those not readmitted on separate lines. The first line sets up a figure in much the same way as we did in the previous
cell. This time we use the Kd E plot function. KDE stands for “kernel density estimate” and is a method for visualising
the distribution of observations on a dataset. It is a bit like a histogram but represents the data using a curve. We
use it twice to add two lines to the figure. The first arguments we pass for each line get a list of all the values of
“readmitted” which satisfy the condition, i.e. `== 0`, or `== 1` as well as values for time in hospital.
The other arguments shade the area under the curve and colour it.

In [ ]:
fig = plt.figure(figsize=(13,7),)
ax=sns.kdeplot(df.loc[(df['readmitted'] == 0),'time_in_hospital'] , color='b',shade=True,bw_adjust=2, label='Not Readmitted')
ax=sns.kdeplot(df.loc[(df['readmitted'] == 1),'time_in_hospital'] , color='r',shade=True,bw_adjust=2 , label='Readmitted')
ax.set(xlabel='Time in Hospital', ylabel='Frequency')
plt.title('Time in Hospital VS. Readmission')
plt.legend()

#### Try…
Now it's your turn. Use the cell below to plot some more of data. As you explore, make a note of anything you see which
is unusual.

In [ ]:
# type your code below...


## Modeling
There are lots more things we could do to this data to improve our predictions still further, but the boss has said we
have to finish this by lunchtime, so let's quickly deploy our algorithm and make some life or death decisions because
work/life balance is important and nobody likes working on their lunch hour, right?

Anyway, onwards! The next thing to do is identify the features we will use and separate out the target variable we are
trying to predict.

In the cell below we made a list of the features we are going to use to train the model. The last two lines in the cell
extract the features from the database into a variable called X. The other line extracts the target feature, i.e. the
one whose value we are learning to predict into a variable y. The use of uppercase X and lowercase y for features and
target variable is a convention you will see quite often.

We are going to use a library called *__Scikit-Learn__* to do our machine learning. Scikit-Learn is a collection of modules
which give us a number of algorithms to try out, and we are going to experiment with two of them. The techniques we are
going to use are *__supervised algorithms__*. We say this because these techniques use data which is labelled with the values
we are trying to predict, so that we know what the output should be. We use this to create a model based on the
relationship between our training variables and these known outputs. _Unsupervised learning_ techniques work without this
knowledge and find relationships within the data independently.

Our task is a classification task in that we are classifying patients based on whether they will or will not be
readmitted to hospital within 30 days.

In [ ]:
#noodling about - TODO: delete me when ready to publish
feature_set1 = [ 'gender', 'age', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
               'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient',
               'number_emergency', 'number_inpatient',  'number_diagnoses', 'max_glu_serum',
               'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide',
               'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol',
               'troglitazone', 'tolazamide',  'insulin', 'glyburide-metformin',
               'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone',
               'change', 'diabetesMed', 'diag_1_cat', 'diag_2_cat', 'diag_3_cat', 'numchange', 'race_AfricanAmerican',
               'race_Asian','race_Caucasian','race_Hispanic','race_Other']

# specify which columns we are going to use as features for our model
feature_set = [ 'gender', 'age', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
               'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient',
               'number_emergency', 'number_inpatient',  'number_diagnoses', 'max_glu_serum',
               'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide',
               'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol',
               'troglitazone', 'tolazamide',  'insulin', 'glyburide-metformin',
               'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone',
               'change', 'diabetesMed', 'diag_1_cat', 'diag_2_cat', 'diag_3_cat', 'numchange', 'race_AfricanAmerican',
               'race_Asian','race_Caucasian','race_Hispanic','race_Other']


X = df[feature_set]
y = df['readmitted']

### Test and Training Data
We need some way of testing the performance of our model. The way we do this is by taking out and reserving a section of
our data as a test set, and then training the model with the rest. We then use it to make predictions based on the test
data and compare those to the actual values to see how well we did. Usually we keep aside 20% of our data for testing
purposes. Scikit-Learn has a module to do this, and we are going to import that and run a function to split the data.
We will end up with 4 sets. 2 sets of features ``X_train`` and ``X_test`` and 2 sets of targets ``y_train`` and``y_test``- the values we are
trying to predict.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

## The decision tree
The first algorithm we are going to try is the decision tree. It is called a decision tree because the way it works
internally is a bit like a flowchart with a treelike structure. At each node of the tree the algorithm finds the most
efficient way of classifying the data. It keeps splitting the dataset at every node until eventually all the data points
at each end of the tree are in the same class.

In the next cell we have imported a decision tree classifier and created an instance of one as a variable called dtree.
The classifier has a function called `fit()` and we pass it all the values from our dataset that we are going to use for
training along with the values for the target variable. The decision tree classifier then makes a model based on the
relationships it finds between our training data and the target values. Once that is done we can create a set of predicted
values by calling ```dtree.predict(X_test)``` We use the test data for this so that we can check it against the actual
values we kept in `y_test`

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

dtree = DecisionTreeClassifier(max_depth=28, criterion = "entropy", min_samples_split=10)
dtree.fit(X_train, y_train)
dtree_predictions=dtree.predict(X_test)

Now we can check how well our model is doing, using some more functions from Scikit-Learn called ``accuracy score()``,
``precision score``, and ``recall score()``. We pass these our known target values, along with the predictions and
print them out in the cell below.

In [ ]:
print("DT-Accuracy is {0:.2f}".format(accuracy_score(y_test, dtree_predictions)))
print("DT-Precision is {0:.2f}".format(precision_score(y_test, dtree_predictions)))
print("DT-Recall is {0:.2f}".format(recall_score(y_test, dtree_predictions)))


Finally we have some results and our accuracy is 89%! Great! Time for pizza! Most pizza places in Southampton don’t
need to be told where our building is on campus...

_Not so fast!_ Accuracy is a simple measure of the number of correct predictions. We have to be a bit careful about relying
on this beacuse if we created our own model which did not bother analysing any of the data and just said no patients were
ever readmitted, then in a dataset where 80% of the patients were not readmitted we will be right 80% of the time!
That is why we need some other metrics and you may notice we have also printed out two others called _precision_ and _recall_.

* __Precision__ is slightly different from accuracy in that it only measures the rate of false positives, i.e. what proportion
of patients who we predicted would be readmitted actually were.

* __Recall__, on the other hand, measures the proportion of people who were actually readmitted, who we correctly identified as such.

So our results are not looking very good despite the rather more impressive looking accuracy figure.

...Except that these metrics don't work quite so well with decision trees, so we will also look at something called
the *__AUC score__*.
Trying to define and understand how awk scores work just before lunch can result in stomach ulcers according to another
study I just made up, so we will just accept that simply put, an AUC score measures how well the model performs when
compared to something that just makes random predictions. An orc score of 0 means a model is not random but is making
incorrect predictions 100% of the time. An orc score 0.5 means the model is right about half the time, i.e. it is making
random predictions. An orc score of 1.0 means model is correct 100% of the time.

In [ ]:
from sklearn.metrics import  roc_auc_score
print('ROC AUC SCORE')
print(roc_auc_score(y_test, dtree.predict_proba(X_test)[:,1]))

Oh dear, still not great. We're only doing slightly better than we would with a dice!

### Enough Trees, Time For a Forest
Another algorithm we could try, which might improve things, is called a *__Random Forest__*. If you are wondering what the
difference between this and a decision tree is, the clue is in the name. A random Forest runs lots of decision trees
and then averages the results. In the code below we have fit a random forest model in much the same way we did for the
decision tree. The result is a slight improvement but still no pizza

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 50, max_depth=25, criterion = "gini", min_samples_split=10)
rf.fit(X_train, y_train)

rf_predict=rf.predict(X_test)

print("rf-Accuracy is {0:.2f}".format(accuracy_score(y_test, rf_predict)))
print("rf-Precision is {0:.2f}".format(precision_score(y_test, rf_predict)))
print("rf-Recall is {0:.2f}".format(recall_score(y_test, rf_predict)))
print("AUC score is {0:.2f}".format(roc_auc_score(y_test, rf.predict_proba(X_test)[:,1])))

Now that is looking a little bit better!
